# Qualitative Assesment of Models



In [19]:
import  pandas as pd
from IPython.display import display, HTML

In [20]:
# Load predictions
gpt_4 = pd.read_csv('outputs/eval_demographics-fewshot_gpt-4-0125-preview_minc-40_maxc-4000_clean.csv')
gpt_4_o = pd.read_csv('outputs/eval_demographics-fewshot_gpt-4o-2024-05-13_minc-40_maxc-4000_clean.csv')
ff = pd.read_csv('outputs/eval_demographics-fewshot_firefunction-v1_minc-40_maxc-4000_clean.csv')

# Load only subset of columns
embeddings = pd.read_parquet('outputs/eval_embeddings_minc-40_maxc-4000.parquet', 
                             columns=['content', 'start_char', 'end_char', 'pmcid', 'section_0', 'section_1'])

# Load annotations
annotations = pd.read_csv('annotations/jerome_pd.csv').drop(columns=['Unnamed: 0'])

In [21]:
def _get_corr_groups(annotations, predictions):
    annotations = annotations.copy()
    annotations = annotations[annotations.pmcid.isin(predictions.pmcid.unique())]
    pred_n_groups = predictions.groupby('pmcid').size()
    n_groups = annotations.groupby('pmcid').size()
    return (n_groups == pred_n_groups)

In [22]:
gpt_4_corr_groups = _get_corr_groups(annotations, gpt_4)
ff_corr_groups = _get_corr_groups(annotations, ff)

# IDs where both got it right
both_right = gpt_4_corr_groups & ff_corr_groups
both_right = both_right.index[both_right]


In [58]:
def _print_context(pmcid, annotation, predictions, embeddings):
    print("PMC ID: ", pmcid)
    subset_rows = ['count', 'diagnosis', 'group_name', 'subgroup_name',
       'male_count', 'female_count', 'age_mean', 'age_minimum', 'age_maximum',
       'age_median']
    
    pred_subset_cols = ['imaging_sample']
    
    if 'assesment_type' in predictions.columns:
        pred_subset_cols += ['assesment_type']
    
    annotation = annotation[(annotation.pmcid == pmcid)]
    annotation = annotation[subset_rows]

    pred = predictions[predictions.pmcid == pmcid]

    for start_char, g in pred.groupby('start_char'):
        print("Context: ")
        print(embeddings[(embeddings.pmcid == pmcid) & (embeddings.start_char == start_char)].content.values[0])

        print("Predictions: ")
        display(g[subset_rows + pred_subset_cols])

    print("Annotation: ")
    display(annotation)

In [24]:
combined = pd.concat([gpt_4_o, gpt_4, ff], keys=['gpt-4o', 'gpt4-turbo', 'firefunction-v1'], names=['model'])

In [25]:
combined

count diagnosis group_name subgroup_name  male_count  \
model                                                                       
gpt-4o          0     15.0       NaN    healthy           NaN         5.0   
                1     23.0       NaN    healthy  participants        18.0   
                2     16.0       NaN    healthy           NaN         9.0   
                3     15.0       NaN    healthy          fMRI         8.0   
                4     45.0       NaN    healthy  participants        18.0   
...                    ...       ...        ...           ...         ...   
firefunction-v1 529   15.0       TSC   patients     reduction         7.0   
                530   15.0       TSC   patients         total         7.0   
                531   15.0       TSC   patients        volume         7.0   
                532   15.0       TSC   patients    target AML         7.0   
                533   15.0       TSC   patients      compared         7.0   

                     female_count  age_mean age_range  age_minimum  \
model                                                                
gpt-4o          0            10.0     31.60     22-54         22.0   
                1             5.0    222.00       NaN          NaN   
                2             7.0       NaN     18-35         18.0   
                3             7.0     22.21       NaN          NaN   
                4            27.0     24.49       NaN          NaN   
...                           ...       ...       ...          ...   
firefunction-v1 529           8.0     43.50     20-60         20.0   
                530           8.0     44.50     20-60         20.0   
                531           8.0     45.50     20-60         20.0   
                532           8.0     46.50     20-60         20.0   
                533           8.0       NaN       NaN          NaN   

                     age_maximum  age_median imaging_sample    pmcid  rank  \
model                                                                        
gpt-4o          0           54.0         NaN            yes  7981326   0.0   
                1            NaN         NaN             no  4330553   0.0   
                2           35.0         NaN             no  3183226   0.0   
                3            NaN         NaN            yes  6195265   0.0   
                4            NaN         NaN            yes  5428836   0.0   
...                          ...         ...            ...      ...   ...   
firefunction-v1 529         60.0        43.0            yes  9308181   NaN   
                530         60.0        44.0            yes  9308181   NaN   
                531         60.0        45.0            yes  9308181   NaN   
                532         60.0        46.0            yes  9308181   NaN   
                533          NaN         NaN            yes  9308181   NaN   

                     start_char  end_char  
model                                      
gpt-4o          0          9579     10553  
                1         10707     11145  
                2         28053     28467  
                3         13097     13950  
                4         26550     27302  
...                         ...       ...  
firefunction-v1 529        5201      8942  
                530        5201      8942  
                531        5201      8942  
                532        5201      8942  
                533        5201      8942  

[1405 rows x 16 columns]

In [26]:
_print_context(both_right[2], annotations, combined, embeddings)

PMC ID:  2525845
Context: 

## Material and methods 
  
### Subjects 
  
Thirty-four healthy, right-handed, heterosexual German subjects (age-range 23–45 years) participated in the study after giving their informed consent. The study was approved by the local Institutional Ethical Review Board and was performed in accordance with the ethical standards laid down in the 1964 Declaration of Helsinki. Men were examined once, women twice in pseudo-randomized order—once 1–3 days after onset of menses (early follicular phase) and once in the midluteal phase adjusted for cycle length (individual cycle length −7 days with a range of ±2 days). Only pre-menopausal women with stable cycle length were included and any form of hormonal treatment was excluded. The phase of the menstrual cycle was confirmed by sex steroid hormone analysis (for details see below). None of the subjects had a history of a serious medical disease, or any neurological or psychiatric illness. Twelve females and 12 males wer

,,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median,imaging_sample
model,,,,,,,,,,,,
gpt-4o,316,24.0,NaN,healthy,final analysis,12.0,12.0,NaN,23.0,45.0,NaN,yes
gpt4-turbo,312,24.0,NaN,healthy,final analysis,12.0,12.0,NaN,23.0,45.0,NaN,yes
firefunction-v1,359,34.0,NaN,healthy,all,12.0,12.0,2345.0,23.0,45.0,2345.0,no


Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
4,24,NaN,healthy,_,12.0,12.0,NaN,NaN,NaN,NaN


In this example, firefunction does well, but is not able to recognize that the final sample was 12 male + 12 female subjects = 24. 
GPT-4, impressively, is able to extract this complex annotation

In [27]:
_print_context(both_right[3], annotations, combined, embeddings)

PMC ID:  2648877
Context: 

## Methods 
  
### Participants 
  
Thirty-three individuals participated in the study: 15 participants with Autism Spectrum Disorder (12 males; 3 females) and 18 non-autistic control participants (13 males; 5 females). Groups were matched on age (ASD   M  : 38 years, SD: 13; control   M  : 32 years, SD: 8;   t  (31) = 1.6,   p   = .13), and IQ (ASD   M  : 119, SD: 14; control   M  : 119, SD: 11;   t  (31) = 0.1,   p   = .93). Full-scale IQ was measured using the Wechsler Adult Intelligence Scale 3rd UK Edition ( ), apart from one control participant for whom IQ was estimated from the National Adult Reading Test (NART;  ). All participants in the ASD group had previously received a diagnosis from an independent clinician according to standard criteria. The Autism Diagnostic Observational Schedule-Generic (ADOS-G,  ), was used to characterise the participants’ level of current functioning. This measure was chosen because all participants were adults; it was t

count                 diagnosis group_name subgroup_name  \
model                                                                           
gpt-4o          133   15.0  Autism Spectrum Disorder   patients           ASD   
                134   18.0                       NaN    healthy       control   
gpt4-turbo      128   15.0  Autism Spectrum Disorder   patients           ASD   
                129   18.0                       NaN    healthy       control   
firefunction-v1 140   33.0  Autism Spectrum Disorder   patients           ASD   
                141   18.0                       NaN    healthy       control   

                     male_count  female_count  age_mean  age_minimum  \
model                                                                  
gpt-4o          133        12.0           3.0      38.0          NaN   
                134        13.0           5.0      32.0          NaN   
gpt4-turbo      128        12.0           3.0      38.0          NaN   
                129        13.0           5.0      32.0          NaN   
firefunction-v1 140        12.0           3.0      38.0         38.0   
                141        13.0           5.0      32.0         32.0   

                     age_maximum  age_median imaging_sample  
model                                                        
gpt-4o          133          NaN         NaN             no  
                134          NaN         NaN             no  
gpt4-turbo      128          NaN         NaN             no  
                129          NaN         NaN             no  
firefunction-v1 140         38.0        38.0            yes  
                141         32.0        32.0            yes

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
7,18,NaN,healthy,_,13.0,5.0,32.0,NaN,NaN,NaN
8,15,Autism Spectrum Disorder,patients,_,12.0,3.0,38.0,NaN,NaN,NaN


Here, FireFunction mistakes the total sample size (33), for the subgroup size (Autism Spectrum Disorder)

In [28]:
_print_context(both_right[4], annotations, combined, embeddings)

PMC ID:  2775905
Context: 

## Materials and methods 
  
### Participants 
  
Twenty-two participants (13M, 9F) took part in the study although four (2M, 2F) were subsequently excluded due to: i) excessive head motion (> 10 mm), ii) an unexpected brain abnormality, iii) chance level performance in the scanner, and iv) an error acquiring the scanning data. The 18 remaining participants were right-handed, native speakers of British English (mean 26.7 years, median 22.5 years, range 18–60 years) without any history of neurological or psychiatric disease. The behavioural and neuroimaging data from the older participant (the one 60 year-old) did not differ qualitatively from the younger participants and therefore was included in all analyses. None had any form of oral or written language impairment or any previous experience with time-compressed speech. None of the participants reported any hearing difficulties, but were not audiometrically screened. In-scanner preliminary testing revealed 

,,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median,imaging_sample
model,,,,,,,,,,,,
gpt-4o,73,18.0,NaN,healthy,NaN,11.0,7.0,26.7,18.0,60.0,22.5,yes
gpt4-turbo,68,18.0,NaN,healthy,NaN,11.0,7.0,26.7,18.0,60.0,22.5,yes
firefunction-v1,119,22.0,NaN,healthy,participants,13.0,9.0,26.7,18.0,60.0,22.0,no


Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
10,18,NaN,healthy,_,NaN,NaN,26.7,18.0,60.0,22.5


Here, FireFunction gets the correct *total* sample size, but not the imaging sample size. 

In [29]:
_print_context(both_right[7], annotations, combined, embeddings)

PMC ID:  3002948
Context: 

## Materials and Methods 
  
### Participants 
  
Fifteen participants were phobic prone (PP) (6 females; mean age 39.2; standard deviation [SD] 7.4) and 15 were eating disorders prone (EDP) (5 females; mean age 34.4; standard deviation [SD] 8.65). The couples enrolled had been together in a committed relationship for the last three years and had been living together for at least one year. To assign the participants to a group, they were assessed with a semi-structured interview  –  and the Personality Meaning Questionnaire (PMQ)   one month before the scanning session. Concordance between the two investigators was 100%. As in our previous study  – , the semi-structured interview was administered independently by two trained investigators who were blind to each other's results. The aim of the semi-structured interview was to assess the key themes characterizing different affective-cognitive styles in the matter of emotional activation, duration and regulatio

count               diagnosis group_name  \
model                                                           
gpt-4o          372   15.0            phobic prone   patients   
                373   15.0  eating disorders prone   patients   
gpt4-turbo      375   15.0            phobic prone   patients   
                376   15.0  eating disorders prone   patients   
firefunction-v1 438   15.0            phobic prone   patients   
                439   15.0  eating disorders prone   patients   

                              subgroup_name  male_count  female_count  \
model                                                                   
gpt-4o          372                      PP         9.0           6.0   
                373                     EDP        10.0           5.0   
gpt4-turbo      375                      PP         9.0           6.0   
                376                     EDP        10.0           5.0   
firefunction-v1 438            phobic prone         6.0           9.0   
                439  eating disorders prone         5.0          10.0   

                     age_mean  age_minimum  age_maximum  age_median  \
model                                                                 
gpt-4o          372      39.2          NaN          NaN         NaN   
                373      34.4          NaN          NaN         NaN   
gpt4-turbo      375      39.2          NaN          NaN         NaN   
                376      34.4          NaN          NaN         NaN   
firefunction-v1 438      39.2         32.0         46.0        39.0   
                439      34.4         26.0         42.0        34.0   

                    imaging_sample  
model                               
gpt-4o          372            yes  
                373            yes  
gpt4-turbo      375            yes  
                376            yes  
firefunction-v1 438             no  
                439             no

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
15,15,NaN,healthy,phobic prone,9.0,6.0,39.2,NaN,NaN,NaN
16,15,NaN,healthy,eating disorders prone,10.0,5.0,34.4,NaN,NaN,NaN


This is a fairly impressive result for both models. They got the correct sample size for each subgroup.

In general, in a large amount of examples, both models extract the correct information.

The column `imaging_sample` is often wrong for short contexsts. It only seems correct when there's a behavioral and non behavioral sample.

In [30]:
_print_context(both_right[17], annotations, combined, embeddings)

PMC ID:  3422286
Context: 

## Methods 
  
### Participants 
  
Twelve subjects (six females) with a mean age of 26 years (range 23–39 years) participated in the first experiment, 14 in the second (seven females, mean age 23 years, range 24–30 years). The participants gave informed written consent to the study, which adhered to the Declaration of Helsinki and was approved by the human subjects committee of the Otto-von-Guericke University Magdeburg. 

All subjects were right-handed as assessed by the Edinburgh Handedness Inventory  . 


Predictions: 


count diagnosis group_name      subgroup_name  male_count  \
model                                                                           
gpt-4o          5    12.0       NaN    healthy   first experiment         6.0   
                6    14.0       NaN    healthy  second experiment         7.0   
gpt4-turbo      9    12.0       NaN    healthy   first experiment         6.0   
                10   14.0       NaN    healthy  second experiment         7.0   
firefunction-v1 8    12.0       NaN    healthy   first experiment         6.0   
                9    14.0       NaN    healthy  second experiment         7.0   

                    female_count  age_mean  age_minimum  age_maximum  \
model                                                                  
gpt-4o          5            6.0      26.0         23.0         39.0   
                6            7.0      23.0         24.0         30.0   
gpt4-turbo      9            6.0      26.0         23.0         39.0   
                10           7.0      23.0         24.0         30.0   
firefunction-v1 8            6.0      26.0         23.0         39.0   
                9            7.0      23.0         24.0         30.0   

                    age_median imaging_sample  
model                                          
gpt-4o          5          NaN             no  
                6          NaN             no  
gpt4-turbo      9          NaN             no  
                10         NaN             no  
firefunction-v1 8         26.0             no  
                9         23.0             no

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
30,14,NaN,healthy,study 2,7.0,7.0,23.0,24.0,30.0,NaN
31,12,NaN,healthy,study 1,6.0,6.0,26.0,23.0,39.0,NaN


In [31]:
_print_context(9564100, annotations, combined, embeddings)

PMC ID:  9564100
Context: 

## Materials and Methods 
  
### Participants. 
  
We recruited healthy, young participants (872 females, 546 males, mean age = 22.39 y, SD = 3.27). Advertising was done mainly at the University of Basel and in local newspapers. The participants were free of any neurological or psychiatric illness, did not take any medication at the time of the experiment (except hormonal contraceptives), and were between ages 18 and 35 y. Physical and mental health was assessed based on standard questionnaires. The experiment was approved by the ethics committee of the Canton of Basel, Switzerland. All participants gave written informed consent before participating in the study. Prior to the analysis, the sample was divided into a discovery sample (  n   = 945, 2/3 of all participants) and a replication sample (  n   = 473, 1/3 of all participants) by randomly assigning participants to one of the samples. Randomization was performed using the Matlab function randperm. There

count diagnosis group_name  \
model                                             
gpt-4o          39  1418.0       NaN    healthy   
                40   945.0       NaN    healthy   
                41   473.0       NaN    healthy   
gpt4-turbo      78  1418.0       NaN    healthy   
                79   945.0       NaN    healthy   
                80   473.0       NaN    healthy   
firefunction-v1 34  1418.0       NaN    healthy   

                                        subgroup_name  male_count  \
model                                                               
gpt-4o          39                            overall       546.0   
                40                          discovery         NaN   
                41                        replication         NaN   
gpt4-turbo      78                              total       546.0   
                79                          discovery         NaN   
                80                        replication         NaN   
firefunction-v1 34  discovery and replication samples       546.0   

                    female_count  age_mean  age_minimum  age_maximum  \
model                                                                  
gpt-4o          39         872.0     22.39         18.0         35.0   
                40           NaN       NaN          NaN          NaN   
                41           NaN       NaN          NaN          NaN   
gpt4-turbo      78         872.0     22.39         18.0         35.0   
                79           NaN       NaN          NaN          NaN   
                80           NaN       NaN          NaN          NaN   
firefunction-v1 34         872.0     22.39         18.0         35.0   

                    age_median imaging_sample  
model                                          
gpt-4o          39         NaN             no  
                40         NaN             no  
                41         NaN             no  
gpt4-turbo      78         NaN             no  
                79         NaN             no  
                80         NaN             no  
firefunction-v1 34        22.0             no

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
423,1418,NaN,healthy,_,546.0,872.0,22.39,18.0,35.0,NaN


In this example, FF actually got closer, bc GPT exracted the discovery sample and replication sample in addition, both of which are subsample of the total. 
Perhaps I can modify the instruction for "imaging_sample" to say "FINAL total".

Alternative both extractions are relatively useful, if incomplete

## Worst examples

For FireFunction model

In [32]:
ff_mean = ff.groupby('pmcid').apply(lambda x: x['count'].mean())
annot_mean = annotations.groupby('pmcid').apply(lambda x: x['count'].mean())
((ff_mean-annot_mean) / annot_mean).abs().sort_values(ascending=False).head(10)

pmcid
5832413    57.687500
3423412     2.000000
7004957     1.941176
5598991     1.387931
3701149     1.221277
3913832     1.129032
7395771     1.000000
8782893     1.000000
3554651     1.000000
4048172     1.000000
dtype: float64

In [33]:
_print_context(5832413, annotations, combined, embeddings)

PMC ID:  5832413
Context: 

## Materials and methods 
  
### Participants 
  
#### Behavior-based visual similarity rating task and conceptual feature generation task 
  
A total of 2846 individuals completed online behavioral tasks using Amazon’s Mechanical Turk ( ). Data from 61 participants were discarded due to technical errors, incomplete submissions, or missed catch trials. Of the remaining 2785 participants, 1185 completed the visual similarity rating task (616 males, 569 females; age range = 18–53; mean age = 30.1), and 1600 completed the semantic feature generation task (852 males, 748 females; age range = 18–58 years; mean age = 31.7). These sample sizes are proportionally in line with those reported by  . Individuals who completed the visual similarity rating task were excluded from completing the feature generation task, and vice versa. All participants provided informed consent and were compensated for their time. Both online tasks were approved by the University of Toront

count diagnosis group_name  \
model                                              
gpt-4o          201  1185.0       NaN    healthy   
                202  1600.0       NaN    healthy   
                203    16.0       NaN    healthy   
gpt4-turbo      201  2785.0       NaN    healthy   
                202    16.0       NaN    healthy   
firefunction-v1 226  2785.0       NaN    healthy   
                227    16.0       NaN    healthy   
                228    16.0       NaN    healthy   

                                                         subgroup_name  \
model                                                                    
gpt-4o          201                      visual similarity rating task   
                202                   semantic feature generation task   
                203                                          fMRI task   
gpt4-turbo      201  behavior-based visual similarity rating task a...   
                202                              brain-based fMRI task   
firefunction-v1 226                                         behavioral   
                227                                               fMRI   
                228                                               fMRI   

                     male_count  female_count  age_mean  age_minimum  \
model                                                                  
gpt-4o          201       616.0         569.0      30.1         18.0   
                202       852.0         748.0      31.7         18.0   
                203         6.0          10.0      23.1         19.0   
gpt4-turbo      201      1468.0        1317.0      30.9         18.0   
                202         6.0          10.0      23.1         19.0   
firefunction-v1 226       616.0         569.0      30.1         18.0   
                227        10.0           6.0      23.1         19.0   
                228        10.0           6.0      23.1         19.0   

                     age_maximum  age_median imaging_sample  
model                                                        
gpt-4o          201         53.0         NaN             no  
                202         58.0         NaN             no  
                203         29.0         NaN            yes  
gpt4-turbo      201         58.0         NaN             no  
                202         29.0         NaN            yes  
firefunction-v1 226         53.0        30.0             no  
                227         29.0        23.0            yes  
                228         29.0        23.0            yes

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
214,16,NaN,healthy,_,6.0,10.0,23.1,19.0,29.0,NaN


Actually, in this example, both models do a good job of filtering the `imaging_sample` from the behavioral sample. FF dobule extracts sample size, however. 

Could add a rule that if two extractions are identical across all rows, a duplicate is removed. Could accidentally remove real equal sample sizes, however.

In [34]:
_print_context(3423412, annotations, combined, embeddings)

PMC ID:  3423412
Context: 

## Materials and Methods 
  
### Subjects 
  
Thirty-six right-handed subjects participated in this study after giving written informed consent, including 14 patients with AD, 8 patients with MCI and 14 healthy controls. This study was approved by the Medical Research Ethics Committee of Xuanwu Hospital. The AD and MCI subjects were recruited from patients who had consulted the memory clinic at Xuanwu Hospital for memory complaints. The healthy elderly controls were recruited from the local community. 

All AD patients underwent a complete physical and neurological examination, standard laboratory tests and an extensive battery of neuropsychological assessments. The diagnosis of AD fulfilled the Diagnostic and Statistical Manual of Mental Disorders 4th Edition criteria for dementia (American Psychiatric Association, 1994), and the National Institute of Neurological and Communicative Disorders and Stroke/Alzheimer Disease and Related Disorders Association (NI

count         diagnosis group_name       subgroup_name  \
model                                                                         
gpt-4o          324   14.0                AD   patients                  AD   
                325    8.0               MCI   patients                 MCI   
                326   14.0               NaN    healthy    healthy controls   
gpt4-turbo      347   36.0               NaN    healthy  total participants   
                348   14.0                AD   patients         AD patients   
                349    8.0               MCI   patients        MCI patients   
                350   14.0               NaN    healthy    healthy controls   
firefunction-v1 380   36.0  AD, MCI, healthy   patients                  AD   
                381   36.0  AD, MCI, healthy   patients                 MCI   
                382   36.0               NaN    healthy             healthy   

                     male_count  female_count  age_mean  age_minimum  \
model                                                                  
gpt-4o          324         NaN           NaN       NaN          NaN   
                325         NaN           NaN       NaN          NaN   
                326         NaN           NaN       NaN          NaN   
gpt4-turbo      347         NaN           NaN       NaN          NaN   
                348         NaN           NaN       NaN          NaN   
                349         NaN           NaN       NaN          NaN   
                350         NaN           NaN       NaN          NaN   
firefunction-v1 380        14.0          22.0      70.5         60.0   
                381         8.0          12.0      72.5         65.0   
                382        14.0          22.0      68.5         60.0   

                     age_maximum  age_median imaging_sample  
model                                                        
gpt-4o          324          NaN         NaN            yes  
                325          NaN         NaN            yes  
                326          NaN         NaN            yes  
gpt4-turbo      347          NaN         NaN            yes  
                348          NaN         NaN            yes  
                349          NaN         NaN            yes  
                350          NaN         NaN            yes  
firefunction-v1 380         80.0        70.0            yes  
                381         85.0        72.0            yes  
                382         80.0        68.0            yes

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
32,8,mild cognitive impairment,patients,MCI,NaN,NaN,NaN,NaN,NaN,NaN
33,14,Alzheimer disease,patients,AD,NaN,NaN,NaN,NaN,NaN,NaN
34,14,NaN,healthy,_,NaN,NaN,NaN,NaN,NaN,NaN


GPT-4 handled this better than FireFunction, by identifying subgroups (total participants), and subgroups. 

In [35]:
_print_context(7004957, annotations, combined, embeddings)

PMC ID:  7004957
Context: 

## Procedure and Methods 
  
### Open fMRI Dataset and Preprocessing 
  
The dataset used in the current study, from the open-fMRI repository , consists of 28 healthy younger adults (YA: M  = 24.39 years, age range = 18–34; nine females) and 24 healthy older adults (OA: M  = 66.95 years, age range = 55–75; nine females). There were no significant differences between groups in terms of intellectual ability, as measured by either education or the Wechsler Test of Adult Reading (Wechsler,  ):   M  : YA = 16.85 vs. OA = 16.38 years;   M  : YA = 43.96/50 vs. OA = 39.75/50. 

We used high-resolution structural images for our analyses (MPRAGE; TR = 1,950 ms; TE = 2.26 ms; FA = 7°; 1-mm isotropic voxel; FOV = 256 mm) and five identical task runs of the dataset (142 volumes for each EPI; 41 interleaved 4-mm slices with no gap; TR = 2,000 ms; TE = 25 ms; FA = 90°; matrix size = 64 × 64; FOV = 256), in which participants were exposed to different levels of auditory and

count diagnosis group_name   subgroup_name  male_count  \
model                                                                    
gpt-4o     382   28.0       NaN    healthy  younger adults        19.0   
           383   23.0       NaN    healthy    older adults        14.0   
gpt4-turbo 411   28.0       NaN    healthy  younger adults        19.0   
           412   23.0       NaN    healthy    older adults        15.0   

                female_count  age_mean  age_minimum  age_maximum  age_median  \
model                                                                          
gpt-4o     382           9.0     24.39         18.0         34.0         NaN   
           383           9.0     66.95         55.0         75.0         NaN   
gpt4-turbo 411           9.0     24.39         18.0         34.0         NaN   
           412           9.0     66.95         55.0         75.0         NaN   

               imaging_sample  
model                          
gpt-4o     382            yes  
           383            yes  
gpt4-turbo 411            yes  
           412            yes

Context: 

## Ethics Statement 
  
The dataset and code used in this study is from a public data repository. The original data collection procedure involving human participants was reviewed and approved by IRB at the University of Southern California. 


Predictions: 


count   diagnosis group_name subgroup_name  male_count  \
model                                                                         
firefunction-v1 460  100.0         NaN    healthy     subgroup1        50.0   
                461   50.0  depression   patients     subgroup2        25.0   
                462   75.0     anxiety   patients     subgroup3        37.0   

                     female_count  age_mean  age_minimum  age_maximum  \
model                                                                   
firefunction-v1 460          50.0      25.5         20.0         30.0   
                461          25.0      30.5         25.0         35.0   
                462          38.0      28.5         22.0         32.0   

                     age_median imaging_sample  
model                                           
firefunction-v1 460        25.0            yes  
                461        30.0             no  
                462        28.0            yes

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
294,28,NaN,healthy,young,19.0,9.0,24.39,18.0,34.0,NaN
295,23,NaN,healthy,old,NaN,NaN,NaN,NaN,NaN,NaN


FF hallucinated results from the wrong context

In [36]:
_print_context(5598991, annotations, combined, embeddings)

PMC ID:  5598991
Context: 

### 1.2 Participants 
  
All patients were recruited from the outpatient clinic or inpatient wards of the Psychiatry Department of the First Affiliated Hospital of Kunming Medical University. Right-handed, 147 MDD patients (50 males and 97 females) met the criteria were recruited in the depression group. The inclusion criteria were as follows: ① The diagnosis of MDD was independently made by two experienced psychiatrists in accordance with the Diagnostic and Statistical Manual of Mental Disorders, fourth edition (DSM-IV, American Psychiatry Association, 1994), ② first episode without a history of antidepressants treatment, ③ be aged between 18–45 years, ④ the total score of 17-item Hamilton Depression Rating Scale (HDRS) was not less than 17, ⑤ right handedness, ⑥ the patients or their legal guardian signed the informed consent form. The exclusion criteria included the following items: ① having a history of Axis I psychiatric disorders. ② having a history of

count diagnosis group_name subgroup_name  male_count  \
model                                                                       
gpt-4o          159  147.0       MDD   patients    depression        50.0   
                160  130.0       NaN    healthy       control        49.0   
gpt4-turbo      190  147.0       MDD   patients    depression        50.0   
                191  130.0       NaN    healthy       control        49.0   
firefunction-v1 177  147.0       MDD   patients    depression        50.0   
                178  130.0       NaN    healthy       control        49.0   

                     female_count  age_mean  age_minimum  age_maximum  \
model                                                                   
gpt-4o          159          97.0       NaN         18.0         45.0   
                160          81.0       NaN          NaN          NaN   
gpt4-turbo      190          97.0       NaN         18.0         45.0   
                191          81.0       NaN          NaN          NaN   
firefunction-v1 177          97.0      30.5         18.0         45.0   
                178          81.0      31.5         18.0         45.0   

                     age_median imaging_sample  
model                                           
gpt-4o          159         NaN            yes  
                160         NaN            yes  
gpt4-turbo      190         NaN            yes  
                191         NaN            yes  
firefunction-v1 177        30.0            yes  
                178        31.0            yes

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
195,52,NaN,healthy,old hc,NaN,NaN,NaN,NaN,NaN,NaN
196,60,NaN,healthy,young hc,NaN,NaN,NaN,NaN,NaN,NaN
197,58,early adult onset depression,patients,eod,NaN,NaN,NaN,NaN,NaN,NaN
198,62,later adult onset depression,patients,lod,NaN,NaN,NaN,NaN,NaN,NaN


Both models did fine, just didn't break groups into finer subsections

In [37]:
_print_context(3701149, annotations, combined, embeddings)


PMC ID:  3701149
Context: 

## Materials and methods 
  
### Participants 
  
Eighty adolescents aged from 12 to 20 years participated in the study. Exclusion criteria included age, the presence of any neurological problem or a diagnosis of schizophrenia or schizoaffective disorder according to DSM-IV-TR criteria. Thirty-two subjects were excluded for head movement exceeding 4.7 mm in any of the 6 directions during the scan sessions (Control group:   N   = 9, AH group:   N   = 6, 22q11.2DS group:   N   = 17). In the Control group, we excluded subjects with maladaptive functioning above the clinical cut-off of the Internalizing and Externalizing scales (  t  -score >64) in the Youth Self-Report and Adult Behavior Checklist (Achenbach,  ,  ) (  N   = 1). After excluding these 33 subjects, the 47 remaining youths were distributed in the following three groups: typically developing adolescents (Control group:   N   = 22), adolescents with transient AHs (AH group:   N   = 12) and adolescent

count                  diagnosis group_name  \
model                                                              
gpt-4o          335   22.0                        NaN    healthy   
                336   12.0              transient AHs   patients   
                337   13.0  22q11.2 Deletion Syndrome   patients   
gpt4-turbo      343   22.0                        NaN    healthy   
                344   12.0            subclinical AHs   patients   
                345   13.0  22q11.2 Deletion Syndrome   patients   
firefunction-v1 394   80.0                        NaN    healthy   
                395   47.0                        NaN    healthy   
                396   22.0                        NaN    healthy   
                397   12.0                        NaN    healthy   
                398   13.0                        NaN    healthy   

                       subgroup_name  male_count  female_count  age_mean  \
model                                                                      
gpt-4o          335          Control        16.0           6.0     16.00   
                336               AH         5.0           7.0     15.97   
                337        22q11.2DS         9.0           4.0     16.14   
gpt4-turbo      343    Control group        16.0           6.0     16.00   
                344         AH group         5.0           7.0     15.97   
                345  22q11.2DS group         9.0           4.0     16.14   
firefunction-v1 394              all        32.0          48.0     16.00   
                395            final        16.0          31.0     16.00   
                396          control        16.0           6.0     16.00   
                397               ah         5.0           7.0     16.00   
                398        22q11.2ds         9.0           4.0     16.10   

                     age_minimum  age_maximum  age_median imaging_sample  
model                                                                     
gpt-4o          335         14.0         18.0         NaN            yes  
                336         13.0         18.0         NaN            yes  
                337         13.0         19.0         NaN            yes  
gpt4-turbo      343          NaN          NaN         NaN            yes  
                344          NaN          NaN         NaN            yes  
                345          NaN          NaN         NaN            yes  
firefunction-v1 394         12.0         20.0        16.0             no  
                395         12.0         20.0        16.0             no  
                396         12.0         20.0        16.0             no  
                397         12.0         20.0        16.0             no  
                398         12.0         20.0        16.0             no

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
55,12,auditory hallucination,patients,ah,5.0,7.0,15.97,NaN,NaN,NaN
56,22,NaN,healthy,_,16.0,6.0,16.00,NaN,NaN,NaN
57,13,22q11.2 Deletion Syndrome,patients,22q11.2DS,9.0,4.0,16.14,NaN,NaN,NaN


FF hallucinated several groups, and did not annotate diagnosis using the correct column

In [38]:
_print_context(3913832, annotations, combined, embeddings)


PMC ID:  3913832
Context: 

## Methods 
  
### Participants 
  
A total of sixty-six female participants took part in this study. Thirty-one individuals with a current diagnosis of AN according to DSM-IV criteria were recruited from the hospital and community services of the South London and Maudsley (SLaM) National Health Service Trust and from an online advertisement on the b-eat website (Beating Eating Disorders — ), the UK's largest eating disorder charity (inpatients = 9, outpatients = 8, daycare patients = 7, community = 7). Twenty-five (81%) were diagnosed as restrictive (AN-R) and six (19%) as binge-purging (AN-BP). Fourteen (45%) reported taking antidepressant (SSRI = 12, SNRI = 1) or anti-anxiety medication. Thirty-five age-matched healthy individuals with no personal or family history of eating disorders were recruited from the community, staff and students of the Institute of Psychiatry, King's College London. Two healthy participants were excluded from further analysis due

count                       diagnosis group_name  \
model                                                                   
gpt-4o          146   31.0                anorexia nervosa   patients   
                147   31.0    anorexia nervosa restrictive   patients   
                148    6.0  anorexia nervosa binge-purging   patients   
                149   31.0                             NaN    healthy   
gpt4-turbo      140   31.0           AN (Anorexia Nervosa)   patients   
                141   33.0                             NaN    healthy   
firefunction-v1 153   66.0                              AN   patients   

                        subgroup_name  male_count  female_count  age_mean  \
model                                                                       
gpt-4o          146                AN         0.0          31.0       NaN   
                147              AN-R         0.0          25.0       NaN   
                148             AN-BP         0.0           6.0       NaN   
                149                HC         0.0          31.0       NaN   
gpt4-turbo      140       AN patients         0.0          31.0       NaN   
                141  Healthy controls         0.0          33.0       NaN   
firefunction-v1 153            female         0.0          66.0       NaN   

                     age_minimum  age_maximum  age_median imaging_sample  
model                                                                     
gpt-4o          146          NaN          NaN         NaN             no  
                147          NaN          NaN         NaN             no  
                148          NaN          NaN         NaN             no  
                149          NaN          NaN         NaN             no  
gpt4-turbo      140          NaN          NaN         NaN             no  
                141          NaN          NaN         NaN             no  
firefunction-v1 153          NaN          NaN         NaN             no

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
79,31,NaN,healthy,_,NaN,31.0,NaN,NaN,NaN,NaN
80,31,anorexia nervosa,patients,_,NaN,31.0,NaN,NaN,NaN,NaN


FF combined healthy and diseased groups

# Few Shot 2

In [43]:

ff2 = pd.read_csv('outputs/eval_demographics-fewshot2_firefunction-v1_minc-40_maxc-4000_clean.csv')
gpt4o2 = pd.read_csv('outputs/eval_demographics-fewshot2_gpt-4o-2024-05-13_minc-40_maxc-4000_clean.csv')

In [44]:
ff2_mean = ff2.groupby('pmcid').apply(lambda x: x['count'].mean())
annot_mean = annotations.groupby('pmcid').apply(lambda x: x['count'].mean())
worst = ((ff2_mean-annot_mean) / annot_mean).abs().sort_values(ascending=False).head(10)

combined2 = pd.concat([gpt_4_o, gpt4o2, ff, ff2], keys=['gpt-4o', 'gpt-4o_fs2', 'ff', 'ff_fs2'], names=['model'])

In [59]:
for i in worst.index:
    _print_context(i, annotations, combined2, embeddings)


PMC ID:  5832413
Context: 

## Materials and methods 
  
### Participants 
  
#### Behavior-based visual similarity rating task and conceptual feature generation task 
  
A total of 2846 individuals completed online behavioral tasks using Amazon’s Mechanical Turk ( ). Data from 61 participants were discarded due to technical errors, incomplete submissions, or missed catch trials. Of the remaining 2785 participants, 1185 completed the visual similarity rating task (616 males, 569 females; age range = 18–53; mean age = 30.1), and 1600 completed the semantic feature generation task (852 males, 748 females; age range = 18–58 years; mean age = 31.7). These sample sizes are proportionally in line with those reported by  . Individuals who completed the visual similarity rating task were excluded from completing the feature generation task, and vice versa. All participants provided informed consent and were compensated for their time. Both online tasks were approved by the University of Toront

count diagnosis group_name                     subgroup_name  \
model                                                                           
gpt-4o     201  1185.0       NaN    healthy     visual similarity rating task   
           202  1600.0       NaN    healthy  semantic feature generation task   
           203    16.0       NaN    healthy                         fMRI task   
gpt-4o_fs2 196  1185.0       NaN    healthy     visual similarity rating task   
           197  1600.0       NaN    healthy  semantic feature generation task   
           198    16.0       NaN    healthy                               NaN   
ff         226  2785.0       NaN    healthy                        behavioral   
           227    16.0       NaN    healthy                              fMRI   
           228    16.0       NaN    healthy                              fMRI   
ff_fs2     185  2785.0   healthy    healthy                        behavioral   
           186    16.0   healthy    healthy                           imaging   

                male_count  female_count  age_mean  age_minimum  age_maximum  \
model                                                                          
gpt-4o     201       616.0         569.0      30.1         18.0         53.0   
           202       852.0         748.0      31.7         18.0         58.0   
           203         6.0          10.0      23.1         19.0         29.0   
gpt-4o_fs2 196       616.0         569.0      30.1         18.0         53.0   
           197       852.0         748.0      31.7         18.0         58.0   
           198         6.0          10.0      23.1         19.0         29.0   
ff         226       616.0         569.0      30.1         18.0         53.0   
           227        10.0           6.0      23.1         19.0         29.0   
           228        10.0           6.0      23.1         19.0         29.0   
ff_fs2     185       616.0         569.0      30.1         18.0         53.0   
           186        10.0           6.0      23.1         19.0         29.0   

                age_median imaging_sample assesment_type  
model                                                     
gpt-4o     201         NaN             no            NaN  
           202         NaN             no            NaN  
           203         NaN            yes            NaN  
gpt-4o_fs2 196         NaN            NaN     behavioral  
           197         NaN            NaN     behavioral  
           198         NaN            NaN        imaging  
ff         226        30.0             no            NaN  
           227        23.0            yes            NaN  
           228        23.0            yes            NaN  
ff_fs2     185        30.0            NaN     behavioral  
           186        23.0            NaN        imaging

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
214,16,NaN,healthy,_,6.0,10.0,23.1,19.0,29.0,NaN


PMC ID:  3423412
Context: 

## Materials and Methods 
  
### Subjects 
  
Thirty-six right-handed subjects participated in this study after giving written informed consent, including 14 patients with AD, 8 patients with MCI and 14 healthy controls. This study was approved by the Medical Research Ethics Committee of Xuanwu Hospital. The AD and MCI subjects were recruited from patients who had consulted the memory clinic at Xuanwu Hospital for memory complaints. The healthy elderly controls were recruited from the local community. 

All AD patients underwent a complete physical and neurological examination, standard laboratory tests and an extensive battery of neuropsychological assessments. The diagnosis of AD fulfilled the Diagnostic and Statistical Manual of Mental Disorders 4th Edition criteria for dementia (American Psychiatric Association, 1994), and the National Institute of Neurological and Communicative Disorders and Stroke/Alzheimer Disease and Related Disorders Association (NI

...

,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
163,21,Severe Childhood Abuse,patients,severe childhood abuse,15.0,6.0,17.5,NaN,NaN,NaN
164,19,NaN,patients,psychiatric controls,9.0,10.0,16.9,NaN,NaN,NaN
165,27,NaN,healthy,_,21.0,6.0,17.5,NaN,NaN,NaN


PMC ID:  9407088
Context: 

### 3.2. Experiment Results 
  
Taking an 8-year-old girl with autism, numbered 50,795 in the Autism Brain Imaging Data Exchange (ABIDE) dataset, as an example, this paper analyzes the custom template in the form of slices. The fMRI data size of the subject is  . After discarding the data images of the first four time points and eliminating the invalid data whose head movement exceeds the limit at two time points, the final effective data are  , and the size of the final built custom template is  , as shown in  . 

The custom template has individual pertinence, that is, it is constructed by using the subjects’ fMRI sequence. Therefore, the constructed custom template is related to the subjects’ age and brain structure, and the subjects are different from each other.   shows the fMRI data of four example subjects with autism at five different time points and the custom built template in which sub50795 is an 8-year-old girl, sub50625 is a 7-year-old boy, sub51

count diagnosis group_name     subgroup_name  male_count  \
model                                                                       
gpt-4o     426     4.0    autism   patients  example subjects         3.0   
gpt-4o_fs2 425     1.0    autism   patients          sub50795         0.0   
           426     1.0    autism   patients          sub50625         1.0   
           427     1.0    autism   patients          sub51581         1.0   
           428     1.0    autism   patients          sub50526         1.0   
ff         507  1114.0    autism   patients     ABIDE dataset       557.0   
           508     1.0    autism   patients   custom template         1.0   
           509     1.0    autism   patients   custom template         0.0   
ff_fs2     402  1114.0    autism   patients            autism       557.0   

                female_count  age_mean  age_minimum  age_maximum  age_median  \
model                                                                          
gpt-4o     426           1.0     32.25          7.0         64.0         NaN   
gpt-4o_fs2 425           1.0      8.00          8.0          8.0         8.0   
           426           0.0      7.00          7.0          7.0         7.0   
           427           0.0     64.00         64.0         64.0        64.0   
           428           0.0     50.00         50.0         50.0        50.0   
ff         507         557.0     10.50          NaN         18.0        10.0   
           508           0.0      8.50          8.0          8.0         8.0   
           509           1.0      7.50          7.0          7.0         7.0   
ff_fs2     402         557.0      8.00          8.0          8.0         8.0   

               imaging_sample assesment_type  
model                                         
gpt-4o     426            yes            NaN  
gpt-4o_fs2 425            NaN        imaging  
           426            NaN        imaging  
           427            NaN        imaging  
           428            NaN        imaging  
ff         507            yes            NaN  
           508            yes            NaN  
           509            yes            NaN  
ff_fs2     402            NaN        imaging

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
415,539,autism spectrum disorder,patients,abide 1,NaN,NaN,NaN,NaN,NaN,NaN
416,521,autism spectrum disorder,patients,abide 2,NaN,NaN,NaN,NaN,NaN,NaN
417,573,NaN,healthy,abide 1,NaN,NaN,NaN,NaN,NaN,NaN
418,593,NaN,healthy,abide 2,NaN,NaN,NaN,NaN,NaN,NaN


In several examples, the fewshot2 prompt combined groups that should be separate, by either repeating the total count for several groups, or just reporting a single group.

Not sure why this happened. Should try to change to prompt to only identify what assesment type the original groups refer to, to exclude behavioral samples and no more.

### Summary

In general, GPT-4 does a much better job of reporting sub-groups and not the total sample size, like we want the output
FF sometimes combines diseased and healthy groups together, either repeating sample size, or reporting only a single total sample size.

It's possible better prompting could help here, but it's also worth looking at and assesing other models that are better than FF. Perhaps models based on LLama3 instead of LLama2 or Mixtral. Should also look at gpt-4-turbo and see if there's a way to motivate it to prevent laziness. Perhaps it can close the gap. 

At the hackathon we should also have a discussion about what a reasonable schema is for information we want to extract. 